In [ ]:
import sys
sys.path.append('../src')
import TCT as TCT
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import openai
import json

# confidential keys for openAI
confi = json.loads(open("confidential_key.json").read())
openai.organization = confi['Organization']
openai.api_key = confi['API_key']


In [ ]:
# Step1: List all the APIs in the translator system
APInames = TCT.list_Translator_APIs()
print(len(APInames))
#print(APInames.keys())

# Step 2: Get metaKG from Translator APIs
metaKG = TCT.get_KP_metadata(APInames) # This only applies to the Translator APIs
#print(metaKG.columns)
#print(metaKG.shape)

metaKG.to_csv('../metaData/metaKG.csv', index=False)
metaKG = pd.read_csv('../metaData/metaKG.csv')
All_predicates = list(set(metaKG['KG_category']))

KG_category = list(set( list(metaKG['Subject'].unique()) + list(metaKG['Object'].unique())))
KG_predicates = list(metaKG['KG_category'].unique())

query_json_temp = TCT.load_json_template()

In [ ]:
question1 = "What genes or proteins interact with gene KRAS?"  #success
question2 = "What drugs may treat acute myeloid leukemia?" #successful running
question3 = "What could be potential targets for ovarian cancer?" 
question4 = "What are the drugs or small moleculaes that target the gene KRAS?" #successful running
question5 = "What diseases occurence with covid-19?"  #successful running
question6 = "What symptoms are associated with long covid?" # no results
question7 = "What genes are associated with aptosis?"  # successful running


question = question7
query_json = str(query_json_temp)


# formatting query text
input_text = "We know the avaiable predicates in the KG are: " + ','.join(list(set(KG_predicates))) + ". We also know the avaiable categories in the KGs are "+ ','.join(list(set(KG_category))) +". We also know a TRAPI message temple is " +query_json+ ". With the question of " + question + " What is the json format of message to represent this question? The following rules for the output: 1) The result must be just a json format with the same structure with template; 2)categories should be replaced from the categories in the KG; 3)predicts can be replaced from the predicates in the KG; 4) can use the name to fill the ids; 4)the output must start with '{' and end with '}', and be a standard json format.  At least one ids should be given and No annotations are needed!"


In [ ]:
query_json_cur = TCT.ask_chatGPT4(input_text)
query_json_cur_clean = TCT.extract_json(query_json_cur)
query_json_cur_clean

In [ ]:
TCT.TRAPI_json_validation(query_json_cur_clean, All_predicates, KG_category)

In [ ]:
print(TCT.find_similar_category(query_json_cur_clean, KG_category))

In [ ]:
n1_predictes = ['biolink:Gene', 'biolink:Protein']

In [ ]:
print(TCT.find_similar_predicates(query_json_cur_clean, All_predicates))

In [ ]:
selected_predicates = ['biolink:has_participant', 'biolink:related_to',
                       'biolink:has_real_world_evidence_of_association_with',
                       'biolink:occurs_together_in_literature_with',
                        'biolink:associated_with',
                       ]



In [ ]:
# predicates and category formatting
query_json_cur_clean['message']['query_graph']['edges']['e1']['predicates'] = selected_predicates
query_json_cur_clean['message']['query_graph']['nodes']['n1']['categories'] = n1_predictes


In [ ]:
# id formatting
query_json_cur_clean = TCT.format_id(query_json_cur_clean)
print(query_json_cur_clean)

In [ ]:
# Step Select APIs
input_node1_category = query_json_cur_clean['message']['query_graph']['nodes']['n0']['categories']
input_node2_category = query_json_cur_clean['message']['query_graph']['nodes']['n1']['categories']

sele_APIs = TCT.select_API(sub_list=input_node1_category,
                           obj_list=input_node2_category,
                           metaKG=metaKG)

print("all relevant APIs in Translator:")
print(sele_APIs)
print(len(sele_APIs))

# get API URLs
API_URLs = TCT.get_Translator_API_URL(sele_APIs, 
                                      APInames)

# Step 5: Query Translator APIs and parse results
result = TCT.parallel_api_query(API_URLs,query_json=query_json_cur_clean, max_workers=len(API_URLs))

# Step 6: Parse results
result_parsed = TCT.parse_KG(result)


In [ ]:
input_node1_id = query_json_cur_clean['message']['query_graph']['nodes']['n0']['ids'][0]

# Step 7: Ranking the results. This ranking method is based on the number of unique primary infores. It can only be used to rank the results with one defined node.
result_ranked_by_primary_infores = TCT.rank_by_primary_infores(result_parsed, input_node1_id)   # input_node1_id is the curie id of the input node, such as "NCBIGene:1017"


In [ ]:
# print results 
print(result_ranked_by_primary_infores.shape)
result_ranked_by_primary_infores.head(5)

In [ ]:
# Step 8: Visualize the results
TCT.visulization_one_hop_ranking(result_ranked_by_primary_infores, result_parsed, 
                                num_of_nodes = 30, input_query = input_node1_id, 
                                fontsize = 10)